# Task 3 : OpenHygrisC Nitrate Data 

# Aim of This Notebook


In this notebook, we want to import the "messstelle.csv" file to our Postgres Database

 we need follwing packages: 

* Pandas 
* Sqlalchemy


In [24]:
#importing the packages

import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import credential_temp as  creds 



# Data location

 First, we need to create the path of our data. 


In [25]:
data_in_dir = r"data/"

gw_station_fname = r"OpenHygrisC_gw-messstelle.csv"

gw_station_pfname = data_in_dir + gw_station_fname

## Read the file
Don't forget to add the "delimiter", normal csv file's delimiter is "," but in our case it's ";", so we need to assign it.

In [26]:
df = pd.read_csv(gw_station_pfname, delimiter=';', encoding='latin1', header='infer')

#df = pd.read_csv(gw_station_pfname, delimiter=';', header='infer')
df.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_6660\3457025364.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(gw_station_pfname, delimiter=';', encoding='latin1', header='infer')


,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gemeinde_name,gwhorizont_id,...,beobachtung_wasserstand,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm
0,71143,40007030,GWM301 Aurelis PortsWuhan,3451XX,56998XX,NaN,privat,05112000,Duisburg,NaN,...,-,Aurelis Real Estate GmbH,Aurelis Real Estate GmbH,1600.0,0.0,75.0,NaN,2136.0,3164.0,1564.0
1,24316,91168806,Südl. Kanaldeckel,457597,5718660,1.0,oeffentlich,05974016,Erwitte,NaN,...,durch LANUV,Stadtwerke Lippstadt ...,Land NRW ...,NaN,NaN,117.0,NaN,7200.0,3233.0,3233.0
2,54650,24180993,PB 13T Paderborn,482108,5729807,1.0,oeffentlich,05774032,Paderborn,NaN,...,durch LANUV,Land NRW ...,Land NRW ...,800.0,0.0,125.0,NaN,3538.0,8282.0,7482.0
3,3458,23164529,54 Delbrueck,4736XX,57354XX,1.0,oeffentlich,05774020,Delbrück,NaN,...,-,"Stadt Delbrueck, WW Ostenland ...","Stadt Delbrueck, WW Ostenland ...",NaN,NaN,NaN,NaN,700.0,8875.0,8875.0
4,64554,106502025,HELP9 BR HELPUP,4785XX,57581XX,1.0,privat,05766056,Oerlinghausen,NaN,...,-,Stadtwerke Oerlinghausen GmbH ...,Stadtwerke Oerlinghausen GmbH ...,3000.0,200.0,400.0,NaN,10800.0,7020.0,4020.0


## data type

e32 and n32 should be int if we want to create geometry columns from them

In [27]:
df.dtypes

sl_nr                             int64
messstelle_id                     int64
name                             object
e32                              object
n32                              object
gw_stockwerk                    float64
grundstueck                      object
gemeinde_id                      object
gemeinde_name                    object
gwhorizont_id                    object
gwhorizont                       object
gwleiter_id                      object
gwleiter                         object
einrichtungsgrund                object
gwk_lage_auf_id                  object
gwk_lage_id                      object
gwk_monitoring_auf_id            object
gwk_monitoring_id                object
messprogramm                     object
turnus_wasserstand               object
freigabe_wstd                    object
freigabe_chemie                  object
freigabe_lage                    object
wasserstandsmessstelle           object
guetemessstelle                  object


## change some data types
* e32 to Int
* n32 to Int
* drop NA value

First step, we must drop the NA value.
Second step is some rows the last two digit are xx which means that the exact location is not avaliable. 
Now we need to replace this xx with two digit. These two xx could be in the range of 00 to 99. 
In here we replace the xx with 50

In [28]:
# drop NA
df = df.dropna(subset=['e32'])
df = df.dropna(subset=['n32'])

In [29]:
df.e32

0        3451XX
1        457597
2        482108
3        4736XX
4        4785XX
          ...  
73054    3212XX
73055    3182XX
73056    3268XX
73057    3471XX
73058    3267XX
Name: e32, Length: 73059, dtype: object

In [30]:
# replace all xx with 50

df.e32 = df['e32'].replace({'xx':'50'}, regex=True)
df.n32 = df['n32'].replace({'xx':'50'}, regex=True)

In [31]:
# Drop rows that still contain non-numeric characters
df = df[~df['e32'].str.contains(r'[^\d]', regex=True)]
df = df[~df['n32'].str.contains(r'[^\d]', regex=True)]

In [32]:
# convert e32 and n32 to int
df['e32'] = df['e32'].astype(int)
df['n32'] = df['n32'].astype(int)

### Now the e32 and n32 are int

In [33]:
df.dtypes

sl_nr                             int64
messstelle_id                     int64
name                             object
e32                               int32
n32                               int32
gw_stockwerk                    float64
grundstueck                      object
gemeinde_id                      object
gemeinde_name                    object
gwhorizont_id                    object
gwhorizont                       object
gwleiter_id                      object
gwleiter                         object
einrichtungsgrund                object
gwk_lage_auf_id                  object
gwk_lage_id                      object
gwk_monitoring_auf_id            object
gwk_monitoring_id                object
messprogramm                     object
turnus_wasserstand               object
freigabe_wstd                    object
freigabe_chemie                  object
freigabe_lage                    object
wasserstandsmessstelle           object
guetemessstelle                  object


In [34]:
filt = df['n32'] < 100
df[filt]

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gemeinde_name,gwhorizont_id,...,beobachtung_wasserstand,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm


In [35]:
df.drop(df[df['e32'] < 100].index, inplace = True)
df.drop(df[df['n32'] < 100].index, inplace = True)

### check externally
in order to check the all rows externally we can write it in csv file

In [36]:
#df.to_csv("test.csv", index = False)

Other columns have no problem and the data is ready to import to Database.

## write in Database
Now we want to import our df to postgres database.

* First we have to connect to our Database URL
* then we can write into the our database. we need to assign in which Schema we want to add our data frame. in this case I have added my data frame to the schema "hygrisc" which has been created before in the PGadmin.

In [37]:
# connect to out credential file

postgresurl = creds.URL


In [38]:
# write your sql url below
engine = sqlalchemy.create_engine(postgresurl)


In [39]:
%time df.to_sql(con=engine, name="messstelle", schema="hygrisc", if_exists="replace", index=False)

CPU times: total: 1.75 s
Wall time: 2.7 s


772

### Create geometry columns

Now everything is fine and our data has been imported to Database successfully. but still someting important is missing.
We have e32 and new columns but in order to map these location correclty and exatly in QGIS, we need geometry information. 

in order to create geometry columns from n32 and e32 the follwing step is required. 



In [40]:
quary = """ ALTER TABLE hygrisc.messstelle ADD COLUMN geom geometry(Point, 25832);
 UPDATE hygrisc.messstelle SET geom = ST_SetSRID(ST_MakePoint(e32, n32), 25832);"""

In [41]:
engine.execute(quary)

Now everything is fine and the data has been imported to our database with geometry columns sucessfully. You can see the data in PGadmin as well. 

## Finish!

That's great, You have imported the table successfully in Postgres Database.